In [1]:
%load_ext autoreload
%autoreload 2

from filemanager import fileset
from cli import cli

fileset_manager = fileset.PDSFileset()

qgis_logs = r'D:\aa_yandexcloud\aa_cloudmbesproceessing\ABP56\gis project'
filesets_folder = r'C:\Data\fs'

In [2]:
# pick qgis log
qgis_log_list = cli.search(qgis_logs, '.csv')

Searching *.csv files in directory:D:\aa_yandexcloud\aa_cloudmbesproceessing\ABP56\gis project
Scanned filesets are: 

0 D:\aa_yandexcloud\aa_cloudmbesproceessing\ABP56\gis project\abp56_mbes_filesets.csv
1 D:\aa_yandexcloud\aa_cloudmbesproceessing\ABP56\gis project\abp56_sbp_filesets.csv


In [3]:
qgis_log_picked = cli.pick_from_list(qgis_log_list)

Please type two numbers that define the range of filesets you like to copy or move
Please type the first number: 
Please type the second number: 
Do you want to add more elements? (Y/N)


Picked elements:
abp56_mbes_filesets.csv


In [4]:
import shutil
import os

def qgislogToFilesets(output_dir, qgis_log):
    
    filesets = []
    svp_filesets = []
    filesets_dict = {}

    with open(qgis_log, 'r') as csv_file:
        file_content = csv_file.read().splitlines()

        header = file_content[0].split(',')
        
        filename_idx = 0
        svp_idx = 0
        fileset_idx = 0
        print(header)
        print(filename_idx)
        for field in header:
            if field == 'Fname' or field == 'fname':
                break
            else:
                filename_idx += 1
        for field in header:
            if field == 'svp_statio':
                break
            else:
                svp_idx += 1
                
        for field in header:
            if field == 'fileset':
                break
            else:
                fileset_idx += 1
    
        for num, line in enumerate(file_content[1:]):
            line_content = line.split(',')
            file_name = line_content[filename_idx]
            fileset_name = line_content[fileset_idx]
            svp_name = line_content[svp_idx]

            if fileset_name in filesets_dict.keys():
                filesets_dict[fileset_name].append((file_name, svp_name))
            else:
                filesets_dict[fileset_name] = []
                filesets_dict[fileset_name].append((file_name, svp_name))
    
    for key in filesets_dict.keys():
        fileset_name = key

        fs = fileset.PDSFileset()
        fs.setname(fileset_name)
        
        for filename, svp_name in filesets_dict[key]:
            fs.addfile(filename=filename, svp_name=svp_name)
        filesets.append(fs)
        svp_fs = fs.create_svp_filesets()
        svp_filesets.append(svp_fs)
    
    
    for fs, svpfs in zip(filesets,svp_filesets):
        fs_dir = os.path.join(output_dir,fs.name)
        
        if not os.path.exists(fs_dir):
            os.mkdir(fs_dir)
            
        print(f'{fs.name} - {len(fs.files)}')
        fs.savefileset(dirpath=fs_dir)
        len_svp = 0
        for svfs in svpfs:
            print(f'{svfs.name} - {len(svfs.files)}')
            svfs.savefileset(dirpath=fs_dir)
            len_svp += len(svfs.files)
        print(len_svp)

In [5]:
qgislogToFilesets(filesets_folder, qgis_log_picked[0])

['fid', 'fname', 'begin', 'end', 'uid', 'tsmp', 'datetime', 'x', 'y', 'svp_statio', 'fname_svps', 'length_km', 'length_nm', 'fileset', 'proc_by', 'status']
0
abp56_id024_Prof2_Pr_utm35n - 65
abp56_id024_Prof2_Pr_utm35n_SVP_002 - 65
65
abp56_id017_Gogland_Tr_utm35n - 42
abp56_id017_Gogland_Tr_utm35n_ABP56008 - 25
abp56_id017_Gogland_Tr_utm35n_ABP56006 - 17
42
abp56_id028_ToVyborg_Tr_utm35n - 96
abp56_id028_ToVyborg_Tr_utm35n_SVP_002 - 96
96
abp56_id018_Gogland_Tr_utm35n - 83
abp56_id018_Gogland_Tr_utm35n_ABP56008 - 83
83
abp56_id027_Gogland_Tr_utm35n - 63
abp56_id027_Gogland_Tr_utm35n_SVP_002 - 63
63
abp56_id033_VyborgPo_Po_utm35n - 413
abp56_id033_VyborgPo_Po_utm35n_SVP_003 - 320
abp56_id033_VyborgPo_Po_utm35n_SVP_002 - 93
413
abp56_id014_ToGogland_Tr_utm35n - 74
abp56_id014_ToGogland_Tr_utm35n_ABP56004 - 74
74
abp56_id030_ToVyborg_Tr_utm35n - 83
abp56_id030_ToVyborg_Tr_utm35n_SVP_002 - 83
83
abp56_id015_Gogland_Tr_utm35n - 96
abp56_id015_Gogland_Tr_utm35n_ABP56004 - 60
abp56_id015_Gog